In [122]:
# imports
import csv
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
import seaborn as sns
import pandas as pd
import os
from prettytable import PrettyTable

In [123]:
# read all storage folders to extract training info, i.e. markets and save it to envs
try:
    training_folders = os.listdir('.') 
except(FileNotFoundError):
    training_folders = []

# while iterating create a wideform dataframe of all csv files
df_settings_dict = {}

for folder in training_folders:
    # if "plot_results" in folder or "dqn_comparisons" in folder:
    #     continue

    # read csv file (if it exists)
    try:
        df_settings_dict[folder] = pd.read_csv('./'+folder+'/log.csv')
    except:
        continue

    # add setting to the df columns
    df_settings_dict[folder].columns = [folder + "_" + str(col) for col in df_settings_dict[folder].columns]
    
# join all dataframes (one per setting)
all_data_df = pd.concat([df for df in df_settings_dict.values()], axis=1)

In [124]:
# columns to calculate mean for table (can be extended here)
mean = ["reward", "grid_coloration"]

def get_top_worst_settings(keys, setting, stats_amount=10, exclude=[]):
    all_stats = {}
    for col in all_data_df.columns:
        if any(key in col for key in keys) and setting in col and not any(ignored_setting in col for ignored_setting in exclude):
            for key in keys:
                if key in col and setting in col:
                    stats_key = setting+"|"+key
                    if stats_key not in all_stats:
                        all_stats[stats_key] = {}

                    if "grid_coloration" in key or "reward" in key:
                        # calculate mean
                        all_stats[stats_key][col] = all_data_df[col].mean()
                    else:
                        all_stats[stats_key][col] = sum(all_data_df[col])
    results = []
    table_headers = []
    for stat, entries in all_stats.items():
        formatted_table_header = " ".join(stat.split("_")).split("|")[1]
        table_headers.append("Top " + formatted_table_header)
        table_headers.append("Worst " + formatted_table_header)
        # Top scores
        top_keys = sorted(entries, key=entries.get, reverse=True)[:stats_amount]
        results.append(top_keys)
        results.append([all_stats[stat][top_key] for top_key in top_keys])
        # Worst scores
        worst_keys = sorted(entries, key=entries.get, reverse=False)[:stats_amount]
        results.append(worst_keys)
        results.append([all_stats[stat][worst_key] for worst_key in worst_keys])

    return results, table_headers

# get_top_worst_settings(["fully_colored","mean_grid_coloration"], ["2-dqn", "2-ppo"], stats_amount=15)
def stats_table(*args):
    stats, headers = get_top_worst_settings(*args)
    stats_table = PrettyTable(['Position'] + headers)
    position = 1
    
    for row, _ in enumerate(stats[0]): # go through all entries
        row_entries = [position]
        # odd number contain values, even numbers contain settings
        # stats[1] contains values
        for column, stat in enumerate(stats):
            if column % 2 != 0:
                continue
            short_key = " ".join(stat[row].split("_")[0].split("-"))
            row_entries.append("{:0.3f}".format(stats[column+1][row])+" \n "+ short_key + " \n ")
        
        if row_entries:
            stats_table.add_row(row_entries)   
            position += 1
    print(stats_table)

# Stats Tables

## Top PPO Runs

In [125]:
stats_table(["fully_colored","mean_grid_coloration"], "2-ppo", 5)

+----------+----------------------------------------------------+---------------------+----------------------------------------------------+----------------------------+
| Position |                 Top fully colored                  | Worst fully colored |              Top mean grid coloration              | Worst mean grid coloration |
+----------+----------------------------------------------------+---------------------+----------------------------------------------------+----------------------------+
|    1     |                     3658.000                       |       90.000        |                       0.998                        |           0.520            |
|          |          2 ppo mixed competitive sm goal           |       2 ppo dr      |             2 ppo mixed competitive am             |          2 ppo dr          |
|          |                                                    |                     |                                                    |          

Thoughts:
- worst mean grid coloration all contain cooperative modes -> worst case scenario, agents do not learn 
- worst fully colored entries all contain cooperative modes again! -> only here sm common  
- competitive is always included when it comes to the best results since coloring a field here is rather easy (no reset!) 

## Top PPO Runs without competitive mode

In [126]:
stats_table(["fully_colored","mean_grid_coloration"], "2-ppo", 5, ["competitive"])

+----------+----------------------------------------+---------------------+----------------------------------------+----------------------------+
| Position |           Top fully colored            | Worst fully colored |        Top mean grid coloration        | Worst mean grid coloration |
+----------+----------------------------------------+---------------------+----------------------------------------+----------------------------+
|    1     |               1580.000                 |       90.000        |                 0.793                  |           0.520            |
|          |              2 ppo mixed               |       2 ppo dr      |              2 ppo mixed               |          2 ppo dr          |
|          |                                        |                     |                                        |                            |
|    2     |                443.000                 |       90.000        |                 0.628                  |        

Thoughts:
- without the competitive mode the mixed setting seems to overtake grid coloration with 80 - 55%
- 2 ppo mixed is far better than the rest, regarding only the amount of environment goals 

## Top DQN Runs

In [127]:
stats_table(["fully_colored","mean_grid_coloration"], "2-dqn", 5)

+----------+----------------------------------+---------------------------+----------------------------------+----------------------------+
| Position |        Top fully colored         |    Worst fully colored    |     Top mean grid coloration     | Worst mean grid coloration |
+----------+----------------------------------+---------------------------+----------------------------------+----------------------------+
|    1     |            2358.000              |          51.000           |              0.981               |           0.507            |
|          |     2 dqn mixed competitive      |       2 dqn mixed am      |     2 dqn mixed competitive      |     2 dqn am no reset      |
|          |                                  |                           |                                  |                            |
|    2     |             96.000               |          53.000           |              0.561               |           0.507            |
|          |  2 dqn 

## Top DQN Runs without competitive

In [128]:
stats_table(["fully_colored","mean_grid_coloration"], "2-dqn", 5, ["competitive"])

+----------+----------------------------------+---------------------------+----------------------------------+----------------------------+
| Position |        Top fully colored         |    Worst fully colored    |     Top mean grid coloration     | Worst mean grid coloration |
+----------+----------------------------------+---------------------------+----------------------------------+----------------------------+
|    1     |             96.000               |          51.000           |              0.561               |           0.507            |
|          |  2 dqn am goal no reset no debt  |       2 dqn mixed am      |          2 dqn sm goal           |     2 dqn am no reset      |
|          |                                  |                           |                                  |                            |
|    2     |             72.000               |          53.000           |              0.530               |           0.507            |
|          |        